<a href="https://colab.research.google.com/github/NMPau/Project/blob/main/PDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import DivIcon
import warnings
import scipy as sp
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import seaborn as sns
sns.set_style('white')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
cd DA

/content/drive/My Drive/Colab Notebooks/DA


In [ ]:
cd PDA

[Errno 2] No such file or directory: 'PDA'
/content/drive/My Drive/Colab Notebooks/DA


In [ ]:
!ls

airlines.csv  flights.csv    L_AIRPORT_ID.csv
airports.csv  L_AIRPORT.csv  PDA.ipynb


In [ ]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv', low_memory = False)
aircode1 = pd.read_csv('L_AIRPORT.csv')
aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

Проверка дубликатов

In [ ]:
print(f'Duplicates in df: {flights.iloc[:,1:].duplicated().sum()}')

Duplicates in df: 0


Форматирование кодов аэропортов

In [ ]:
aircode1 = aircode1.reset_index()
aircode2 = aircode2.reset_index()
aircodes = pd.merge(aircode1,aircode2,on='Description')
aircode_dict = dict(zip(aircodes['Code_y'].astype(str),aircodes['Code_x']))

In [ ]:
flights['ORIGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].values.astype(str)
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

for i in range(len(flights)):
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)
for i in range(len(flights)):
    if len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

изменение формата времени

In [ ]:
import datetime
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

In [ ]:
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)

In [ ]:
flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_heure)
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_heure)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_heure)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_heure)

In [ ]:
flights.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,00:05:00,23:54:00,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,04:30:00,04:08:00,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,00:10:00,00:02:00,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,07:50:00,07:41:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,00:20:00,00:18:00,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,08:06:00,08:11:00,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,00:20:00,00:15:00,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,08:05:00,07:56:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,00:25:00,00:24:00,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,03:20:00,02:59:00,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
corr = flights.corr(method='pearson')

Для построения модели для прогнозирования будем использовать опоздание по колонке ARRIVAL_DELAY

In [ ]:
corr['ARRIVAL_DELAY']

In [ ]:
flights.columns

Убираем ненужные колонки

In [ ]:
flights = flights.drop(['YEAR', 'DAY', 'FLIGHT_NUMBER','SCHEDULED_TIME','DIVERTED', 'CANCELLED', 'TAIL_NUMBER', 'AIR_TIME', 'CANCELLATION_REASON', 'DISTANCE'], axis = 1)

Проверяем, есть ли данные с пропущенными значениями

In [ ]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,WEATHER_DELAY,4755640,18.275040
1,AIRLINE_DELAY,4755640,18.275040
2,SECURITY_DELAY,4755640,18.275040
3,AIR_SYSTEM_DELAY,4755640,18.275040
4,LATE_AIRCRAFT_DELAY,4755640,18.275040
5,ARRIVAL_DELAY,105071,98.194371
6,ELAPSED_TIME,105071,98.194371
7,WHEELS_ON,92513,98.410178
8,TAXI_IN,92513,98.410178
9,ARRIVAL_TIME,92513,98.410178


Удаляем строки с пропущенными значениями (для нужных переменных уровень заполненности начинаеется с 98% это хорошо - не будем заполнять средним значением). По переменным с причинами задержки - пропуски заполним нулевым значением (будем считать, что была иная причина задержки)

Удаляем пустые значения

In [ ]:
flights.dropna(subset=['ARRIVAL_DELAY','ELAPSED_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','WHEELS_OFF','TAXI_OUT','DEPARTURE_DELAY','DEPARTURE_TIME'],inplace=True)

In [ ]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,WEATHER_DELAY,4650569,18.611087
1,AIRLINE_DELAY,4650569,18.611087
2,SECURITY_DELAY,4650569,18.611087
3,AIR_SYSTEM_DELAY,4650569,18.611087
4,LATE_AIRCRAFT_DELAY,4650569,18.611087
5,ARRIVAL_DELAY,0,100.000000
6,ARRIVAL_TIME,0,100.000000
7,SCHEDULED_ARRIVAL,0,100.000000
8,TAXI_IN,0,100.000000
9,WHEELS_ON,0,100.000000


Заполняем пропущенные значения нулем

In [ ]:
flights = flights.fillna(0)

In [ ]:
flights.head(5)

,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,ELAPSED_TIME,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,1,4,AS,ANC,SEA,00:05:00,23:54:00,-11.0,21.0,15.0,194.0,404.0,4.0,04:30:00,04:08:00,-22.0,0.0,0.0,0.0,0.0,0.0
1,1,4,AA,LAX,PBI,00:10:00,00:02:00,-8.0,12.0,14.0,279.0,737.0,4.0,07:50:00,07:41:00,-9.0,0.0,0.0,0.0,0.0,0.0
2,1,4,US,SFO,CLT,00:20:00,00:18:00,-2.0,16.0,34.0,293.0,800.0,11.0,08:06:00,08:11:00,5.0,0.0,0.0,0.0,0.0,0.0
3,1,4,AA,LAX,MIA,00:20:00,00:15:00,-5.0,15.0,30.0,281.0,748.0,8.0,08:05:00,07:56:00,-9.0,0.0,0.0,0.0,0.0,0.0
4,1,4,AS,SEA,ANC,00:25:00,00:24:00,-1.0,11.0,35.0,215.0,254.0,5.0,03:20:00,02:59:00,-21.0,0.0,0.0,0.0,0.0,0.0


Найти аэропорт с минимальной задержкой вылета

In [ ]:
a0 = flights.loc[flights['DEPARTURE_DELAY'] == flights['DEPARTURE_DELAY'].min()][['ORIGIN_AIRPORT']]

In [ ]:
AIRPORT_min_delay = airports.loc[airports['IATA_CODE'] == a0['ORIGIN_AIRPORT'].values[0]][['AIRPORT']]

In [ ]:
AIRPORT_min_delay

,AIRPORT
9,Adak Airport


Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

In [ ]:
LA = airports.loc[airports['AIRPORT'] == 'Los Angeles International Airport'][['IATA_CODE']]

In [ ]:
b = flights.loc[flights['DESTINATION_AIRPORT'] == LA['IATA_CODE'].values[0]][['AIRLINE', 'ARRIVAL_DELAY']]

In [ ]:
b1 = b.groupby(['AIRLINE']).mean()
b1

,ARRIVAL_DELAY
AIRLINE,
AA,2.122856
AS,-0.525700
B6,-0.022152
DL,1.398882
F9,10.288235
HA,-4.653317
MQ,18.302469
NK,17.138697
OO,8.202029


Под самой пунктуальной будем понимать авиакомпанию, чье среднее значение самое близкое к нулю (то есть время минимально отклоняется от графика)

In [ ]:
myNumber = 0
m = min(b1['ARRIVAL_DELAY'].values, key=lambda x:abs(x-myNumber))

In [ ]:
ar = b1.loc[b1['ARRIVAL_DELAY'] == m]

In [ ]:
PUNCTUAL_AIRLINE = airlines.loc[airlines['IATA_CODE'] == ar.index[0]][['AIRLINE']]

In [ ]:
PUNCTUAL_AIRLINE

,AIRLINE
4,JetBlue Airways


Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [ ]:
TO = flights[['ORIGIN_AIRPORT', 'TAXI_OUT']]
TI = flights[['DESTINATION_AIRPORT', 'TAXI_IN']]
to = TO.groupby(['ORIGIN_AIRPORT']).mean()
ti = TI.groupby(['DESTINATION_AIRPORT']).mean()

In [ ]:
taxi = to.assign(TAXI_IN = ti.TAXI_IN)
taxi = taxi.assign(TAXI_ALL = taxi.TAXI_OUT + taxi.TAXI_IN)

In [ ]:
taxi

,TAXI_OUT,TAXI_IN,TAXI_ALL
ORIGIN_AIRPORT,,,
ABE,13.325223,3.968635,17.293858
ABI,9.182418,4.003309,13.185727
ABQ,11.948608,5.573155,17.521763
ABR,16.502046,4.679945,21.181991
ABY,12.246849,3.429022,15.675871
...,...,...,...
WRG,7.759547,3.704067,11.463615
WYS,11.903846,4.666667,16.570513
XNA,14.605457,5.102742,19.708200


In [ ]:
max_TAXi = taxi.loc[taxi['TAXI_ALL'] == taxi['TAXI_ALL'].max()]

In [ ]:
max_TAXi_AIRPORT = airports.loc[airports['IATA_CODE'] == max_TAXi.index[0]][['AIRPORT']]

In [ ]:
max_TAXi_AIRPORT

,AIRPORT
182,LaGuardia Airport (Marine Air Terminal)


Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

In [ ]:
!pip install -U lightautoml

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import f1_score, log_loss, roc_auc_score, roc_curve, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_STATE = 777
df_train, df_test = train_test_split(flights, test_size=0.995, random_state = RANDOM_STATE)

In [ ]:
y_train = df_train['ARRIVAL_DELAY']
y_test = df_test['ARRIVAL_DELAY']

In [ ]:
task = Task(name = 'reg')
roles = {'target': 'ARRIVAL_DELAY'}
automl = TabularAutoML(task=task, timeout = 600, memory_limit = 10)
oof_pred = automl.fit_predict(df_train, roles = roles)

INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.
INFO:lightautoml.automl.presets.base:Task: reg

INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:- time: 600.00 seconds
INFO:lightautoml.automl.presets.base:- CPU: 4 cores
INFO:lightautoml.automl.presets.base:- memory: 10 GB

INFO:lightautoml.reader.base:Train data shape: (28570, 21)



BrokenProcessPool: ignored

In [ ]:
df_test = df_test.drop('ARRIVAL_DELAY', axis =1 )
df_train = df_train.drop('ARRIVAL_DELAY', axis =1 )

In [ ]:
y_train_score = automl.predict(df_train).data
y_test_score = automl.predict(df_test).data

In [ ]:
print('Gini_train:', 2*roc_auc_score(y_train, y_train_score)-1)
print('Gini_test:', 2*roc_auc_score(y_test, y_test_score)-1)

In [ ]:
curve1 = roc_curve(y_train, y_train_score)
curve2 = roc_curve(y_test, y_test_score)

plt.figure(figsize =(5,3.5))
plt.plot([0,1], label='Baseline',linestyle='--')
plt.plot(curve1[0], curve1[1], label = 'train')
plt.plot(curve2[0], curve2[1], label = 'test')
plt.title('ROC curve')
plt.xlabel('TPR')
plt.ylabel('FPR')
plt.legend(loc = 4)
plt.show()